https://christophm.github.io/interpretable-ml-book/rulefit.html

https://github.com/christophM/rulefit

https://towardsdatascience.com/interpretable-machine-learning-in-10-minutes-with-rulefit-and-scikit-learn-da9ebb925795



In [30]:
%pip install pyarrow git+https://github.com/christophM/rulefit.git

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
  Cloning https://github.com/christophM/rulefit.git to /tmp/pip-req-build-il0r3gfc
  Running command git clone -q https://github.com/christophM/rulefit.git /tmp/pip-req-build-il0r3gfc
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [19]:
sklearn.__version__

'0.24.1'

In [3]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from plotnine import *
import seaborn as sns

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import statsmodels.api as sm

from rulefit import RuleFit

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

print('\nok')


ok


In [4]:
raw_df = pd.read_feather('data/analysis_df.feather').set_index('id')
raw_df

,is_retracted,publication_year,cited_by_count,ref_length,journal_id,journal_name,n_authors,abstract_indexed,has_doi,has_pmid,concept0,concept1,disrupt,focal_refs,len_I,len_J,novelty_title,novelty_abstract,n_citing
id,,,,,,,,,,,,,,,,,,,
W1000085140,0,2015,8,21,V17140621,Clinical Biochemistry,8,1,1,1,Medicine,None,0.000000,21.0,4.0,4.0,NaN,NaN,8.0
W1000238170,0,2015,23,34,V2764928002,Redox biology,7,1,1,1,Chemistry,Cell biology,0.629630,34.0,22.0,5.0,1.083342,0.644107,27.0
W1000402150,0,2015,1,1,V2764481192,Procedia. Economics and finance,4,1,1,0,Business,None,1.000000,1.0,1.0,0.0,NaN,NaN,1.0
W10005641,0,2013,40,47,V188605413,Carbon,7,1,1,0,None,Optoelectronics,-0.459459,47.0,10.0,27.0,NaN,NaN,37.0
W1000768039,0,2015,50,32,V127530304,Journal of Materials Processing Technology,5,1,1,0,None,None,0.000000,32.0,25.0,25.0,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W99822253,1,2012,4,3,V125754415,Proceedings of the National Academy of Science...,7,0,1,1,Medicine,Immunology,1.000000,3.0,3.0,0.0,NaN,NaN,3.0
W998644059,0,2015,4,47,V186231457,Agroforestry Systems,3,1,1,0,Geography,Socioeconomics,-0.500000,47.0,1.0,3.0,NaN,NaN,4.0
W998753227,0,2015,16,38,V186231457,Agroforestry Systems,3,1,1,0,None,None,-0.333333,38.0,6.0,12.0,NaN,NaN,18.0


In [33]:
raw_df.describe()

,is_retracted,publication_year,cited_by_count,ref_length,n_authors,abstract_indexed,has_doi,has_pmid,disrupt,focal_refs,len_I,len_J,novelty_title,novelty_abstract,n_citing
count,"113,937.000","113,937.000","113,937.000","113,937.000","113,937.000","113,937.000","113,937.000","113,937.000","97,338.000","113,937.000","113,782.000","113,782.000","9,760.000","9,702.000","113,782.000"
mean,0.084,"2,014.158",20.989,31.630,4.936,0.871,0.971,0.593,-0.210,31.630,7.032,14.223,0.937,0.558,21.255
std,0.277,1.976,68.318,31.870,14.337,0.335,0.167,0.491,0.622,31.869,39.776,38.744,0.146,0.230,68.713
min,0.000,"2,011.000",0.000,0.000,0.000,0.000,0.000,0.000,-1.000,0.000,0.000,0.000,0.022,-0.000,0.000
25%,0.000,"2,012.000",2.000,12.000,2.000,1.000,1.000,0.000,-0.692,12.000,1.000,1.000,0.867,0.392,2.000
50%,0.000,"2,014.000",9.000,27.000,4.000,1.000,1.000,1.000,-0.333,27.000,3.000,5.000,0.969,0.523,9.000
75%,0.000,"2,016.000",23.000,43.000,6.000,1.000,1.000,1.000,0.143,43.000,7.000,15.000,1.046,0.719,23.000
max,1.000,"2,017.000","9,992.000","1,355.000","2,864.000",1.000,1.000,1.000,1.000,"1,355.000","8,002.000","3,834.000",1.190,1.152,"9,650.000"


In [34]:
X_names = 'is_retracted publication_year cited_by_count ref_length n_authors'.split()
# X_names = 'is_retracted publication_year'.split()
Y_names = 'novelty_title novelty_abstract disrupt'.split()
wanted_columns = Y_names + X_names
# df = df[df.disrupt.notna()]
df = raw_df[raw_df.ref_length >= 10][wanted_columns].copy()
df.describe()

,novelty_title,novelty_abstract,disrupt,is_retracted,publication_year,cited_by_count,ref_length,n_authors
count,"8,900.000","8,898.000","84,811.000","88,827.000","88,827.000","88,827.000","88,827.000","88,827.000"
mean,0.960,0.578,-0.319,0.073,"2,014.142",25.828,39.981,5.405
std,0.115,0.220,0.527,0.260,1.963,76.264,31.364,16.024
min,0.022,0.043,-1.000,0.000,"2,011.000",0.000,10.000,0.000
25%,0.891,0.417,-0.714,0.000,"2,012.000",5.000,22.000,3.000
50%,0.981,0.538,-0.404,0.000,"2,014.000",13.000,33.000,5.000
75%,1.051,0.736,0.000,0.000,"2,016.000",28.000,48.000,7.000
max,1.190,1.152,1.000,1.000,"2,017.000","9,992.000","1,355.000","2,864.000"


In [35]:
y = df[df.disrupt.notna()].disrupt.values
features = df[X_names].columns
X = df[df.disrupt.notna()][X_names].values
# X = X.as_matrix()


rf = RuleFit()
rf.fit(X, y, feature_names=features)


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.192073269867251, tolerance: 1.6198908761073436
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.9345317305287608, tolerance: 1.6198908761073436
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.2018361069895036, tolerance: 1.6198908761073436
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWa

RuleFit(max_iter=1000,
        tree_generator=GradientBoostingRegressor(learning_rate=0.01,
                                                 max_depth=100,
                                                 max_leaf_nodes=2,
                                                 n_estimators=564,
                                                 random_state=563,
                                                 subsample=0.02178184070719433))

In [38]:
rf.predict(X)

#Inspect rules:

rules = rf.get_rules()

rules = rules[rules.coef != 0].sort_values("support", ascending=False)

In [39]:
#print(*list(rules.rule), sep='\n')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', "{:,.3f}".format)
rules[rules.coef.abs() > .02].head(20)

,rule,type,coef,support,importance
201,is_retracted <= 0.5 & cited_by_count <= 445.5,rule,-0.022,0.920,0.006
955,n_authors > 1.5 & is_retracted <= 0.5,rule,-0.034,0.880,0.011
992,ref_length > 12.5 & ref_length > 21.5 & is_retracted <= 0.5,rule,-0.029,0.751,0.013
1182,n_authors <= 7.5 & ref_length <= 33.5,rule,0.032,0.410,0.016
718,n_authors > 1.5 & ref_length <= 25.5,rule,0.022,0.274,0.010
1134,n_authors > 1.5 & ref_length > 55.5 & cited_by_count > 1.5,rule,-0.027,0.163,0.010
624,cited_by_count <= 19.5 & is_retracted > 0.5,rule,0.022,0.047,0.005
